In [1]:

import vtk
import os
import numpy as np
import itertools
import math, random
import pointnetfunct.data_process_ml as data_process_ml
random.seed = 42
import copy

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torchvision import transforms, utils

import scipy.spatial.distance
# import plotly.graph_objects as go
# import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix



In [2]:
def farthest_point_sampling(points, num_samples):
    """
    Perform farthest point sampling (FPS) on a set of points.
    
    Parameters:
    - points (numpy array): The input point set (Nx3).
    - num_samples (int): The number of samples to be selected.
    
    Returns:
    - sampled_points (numpy array): The sampled points (num_samples x 3).
    """
    N, D = points.shape
    sampled_indices = np.zeros(num_samples, dtype=int)
    distances = np.ones(N) * float('inf')
    farthest = np.random.randint(0, N)
    
    for i in range(num_samples):
        sampled_indices[i] = farthest
        centroid = points[farthest]
        dist = np.sum((points - centroid) ** 2, axis=1)
        distances = np.minimum(distances, dist)
        farthest = np.argmax(distances)
    
    return points[sampled_indices]

In [3]:
import open3d as o3d
def reduce_mesh(target_mesh, reference_mesh):
    # Find common vertices between the two meshes
    output_mesh = copy.deepcopy(target_mesh)
    target_vertices = set(tuple(vertex) for vertex in np.asarray(target_mesh.vertices))
    target_vertices_list = [tuple(vertex) for vertex in np.asarray(target_mesh.vertices)]
    index_dict = {item: index for index, item in enumerate(target_vertices_list)}
    reference_vertices = set(tuple(vertex) for vertex in np.asarray(reference_mesh.vertices))

    common_vertices = target_vertices.intersection(reference_vertices)
    #print(len(list(common_vertices)))
    list_mine = [index_dict[element_to_find] for element_to_find in list(common_vertices)]

    #print(list_mine)

    # Filter out vertices and triangles based on common vertices
    output_mesh.remove_vertices_by_index(list_mine)

    return output_mesh

def mesh_to_point_cloud(file_path,points = 10000):
    """
    Convert a mesh object to a point cloud object.
    """
    mesh = o3d.io.read_triangle_mesh(file_path)
    point_cloud = mesh.sample_points_uniformly(number_of_points=points) # Adjust number_of_points as needed
    return point_cloud

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
import vtk
import numpy as np

def vtp_to_mesh(file_path):
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    polydata = reader.GetOutput()

    vertices = np.array(polydata.GetPoints().GetData())
    polygons = np.array(polydata.GetPolys().GetData())

    # Extract vertices
    vertices = vertices.reshape(-1, 3)

    # Extract faces
    faces = polygons.reshape(-1, 4)[:, 1:]  

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(faces)
    return mesh

def vtp_to_point_cloud(file_path,points = 10000):
    mesh = o3d.geometry.TriangleMesh()
    if ".vtp" not in file_path:
        mesh = o3d.io.read_triangle_mesh(file_path)
    else:
        mesh = vtp_to_mesh(file_path)
        
    #o3d.visualization.draw_geometries([mesh])
    point_cloud= mesh.sample_points_poisson_disk(number_of_points=points) 
    
    # point_cloud_total = mesh.sample_points_uniformly(number_of_points=len(mesh.vertices))
    # if len(mesh.vertices) < points:
    #     point_cloud_total = mesh.sample_points_uniformly(number_of_points=points)        
    
    # point_cloud_total = np.asarray(point_cloud_total.points)
    # point_cloud_points = farthest_point_sampling(point_cloud_total, points)
    
    # point_cloud = o3d.geometry.PointCloud()
    # point_cloud.points = o3d.utility.Vector3dVector(point_cloud_points)
    # print(len(point_cloud.points))

    return point_cloud

def vtp_to_point_cloud_cutvessel(vessel_file_path,cut_file_path, points = 10000):
    """
    Convert a mesh object to a point cloud object.
    """
    if ".vtp" not in vessel_file_path:
        vessel_mesh = o3d.io.read_triangle_mesh(vessel_file_path)
    else:
        vessel_mesh = vtp_to_mesh(vessel_file_path)
    if ".vtp" not in cut_file_path:
        cut_mesh = o3d.io.read_triangle_mesh(cut_file_path)
    else:
        cut_mesh = vtp_to_mesh(cut_file_path)
    
    box_mesh = cut_mesh.get_axis_aligned_bounding_box()
    reduced_mesh =  vessel_mesh.crop(box_mesh) 
    result_mesh = reduce_mesh(vessel_mesh,reduced_mesh)
    #print(cut1_mesh,result_mesh)
    # if vessel_mesh.vertices == result_mesh.vertices:
    #     return False
    point_cloud = result_mesh.sample_points_uniformly(number_of_points=points) # Adjust number_of_points as needed
                  
    #o3d.visualization.draw_geometries([result_mesh])
    #o3d.visualization.draw_geometries([reduced_mesh])
    
    # point_cloud_total = result_mesh.sample_points_uniformly(number_of_points=len(result_mesh.vertices))
    # if len(result_mesh.vertices) < points:
    #     point_cloud_total = result_mesh.sample_points_uniformly(number_of_points=points) 
    

    # point_cloud_total = result_mesh.sample_points_uniformly(number_of_points=len(result_mesh.vertices))
    # point_cloud_total = np.asarray(point_cloud_total.points)
    # point_cloud_points = farthest_point_sampling(point_cloud_total, points)
    
    # point_cloud = o3d.geometry.PointCloud()
    # point_cloud.points = o3d.utility.Vector3dVector(point_cloud_points)
    
    return point_cloud



test case

In [5]:
print(torch.__version__)

2.2.0+cu121


In [6]:
# modelcov_vtp = vtp_to_point_cloud("./test_model/SNF00000036_02.vtp")
# o3d.visualization.draw_geometries([modelcov_vtp])

In [7]:
# print(np.asarray(modelcov_vtp.points))

In [8]:
#modelcov_vtp = vtp_to_point_cloud_cutvessel("./test_model/ANSYS_UNIGE_09.vtp","./test_model/ANSYS_UNIGE_09_dome.vtp")
# modelcov_vtp = vtp_to_point_cloud("./test_model/ANSYS_UNIGE_09_dome.vtp")
# modelcov_vtp = vtp_to_mesh("./test_model/ANSYS_UNIGE_09_dome.vtp")
# mesh_vessel_vtp = vtp_to_mesh("./test_model/ANSYS_UNIGE_09.vtp")
# mesh_vtp = vtp_to_mesh("./test_model/ANSYS_UNIGE_09_cut1.vtp")

# o3d.visualization.draw_geometries([modelcov_vtp])


In [9]:
# modelcov_vtp = vtp_to_point_cloud("./test_model/ANSYS_UNIGE_09_dome.vtp")
# o3d.visualization.draw_geometries([modelcov_vtp])

In [10]:
# modelcov_vtp = vtp_to_point_cloud_cutvessel("./test_model/C0040.vtp","./test_model/C0040_dome.vtp")
# o3d.visualization.draw_geometries([modelcov_vtp])

load the dataset

In [11]:
root = "..\..\msc_data\models-v1.0\models"
IA = "aneurysms\\remeshed\\area-001"
Vessel = "vessels\\remeshed\\area-001"
IA_root = os.path.join(root,IA)
Vessel_root = os.path.join(root,Vessel)
list1 = os.listdir(Vessel_root)
list2 = os.listdir(IA_root)

In [12]:
morpho_path = ".\AneuX\data-v1.0\data\morpho-per-cut.csv"
patient_path = ".\AneuX\data-v1.0\data\clinical.csv"
morpho_data_patient = data_process_ml.read_and_combine_data(morpho_path,patient_path)
merged_dataset = data_process_ml.encode_column(morpho_data_patient)
merged_dataset = data_process_ml.drop_columns(merged_dataset)
morpho_data_cut1,morpho_data_dome = data_process_ml.output_cut1anddome(merged_dataset)

d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(['source_x',"cuttype","dataset"], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [13]:
morpho_data_patient[morpho_data_patient["cuttype"] == "cut1"]

,source_x,dataset,cuttype,curvature-gauss--L2N,curvature.1-gauss-H,curvature.2-gauss-L2N,curvature.3-gauss-L2NCH,curvature.4-mean--L2N,curvature.5-mean-H,curvature.6-mean-L2N,...,zmi_surf.136-energy-n16,zmi_surf.137-energy-n17,zmi_surf.138-energy-n18,zmi_surf.139-energy-n19,zmi_surf.140-energy-n20,status,location,side,sex,age
0,hug2016,p043_HAARCREcDAAQDQcbHgANDRQM,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45.5701,47.4192,49.5589,52.4916,54.9455,unruptured,ICA oph,left,female,64.2
4,hug2016,p044_BBMdFxESDBMcEwcVBhMBExQC,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.1663,44.2741,46.3791,47.4028,49.1238,unruptured,VA V4,left,female,72.7
8,hug2016,p046_FwQADBEGGwQBGwMdHwAADBAK_RICA,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.6116,39.7437,41.7172,44.1529,45.8037,unruptured,MCA bif,right,male,50.9
12,hug2016,p091_ABMXCAMBFT4eAAgABwUQFxAR_LICA,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40.9813,43.2989,45.1772,47.5794,49.8087,ruptured,ICA pcom,left,female,47.3
16,hug2016,p092_ABMXCAMBFT4eAAgABwUQFxAR_RICA,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.152,58.0832,59.6127,60.9068,62.4867,unruptured,ICA oph,right,female,47.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756,aneurisk,C0095,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,35.0355,37.64,40.1607,42.3614,43.8509,unruptured,BA tip,midline,female,43.0
2760,aneurisk,C0096,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37.8419,40.0753,41.8086,43.2362,45.5748,ruptured,BA tip,midline,male,67.0
2764,aneurisk,C0097,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41.9626,44.2732,47.1096,50.0711,52.9587,ruptured,ICA pcom,left,female,84.0
2768,aneurisk,C0098,cut1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.4211,40.5627,42.223,43.9946,46.1836,ruptured,MCA bif,right,female,59.0


In [14]:
import pandas as pd
from pointnetfunct.PointNet_dataset import Aneux_Dataset_load,Aneux_Dataset_save
from pointnetfunct.PointNet_dataset import Aneuxmodel_Dataset_ppd
df = pd.DataFrame()
# Aneuxmodel_Dataset_train_ppd = Aneuxmodel_Dataset_ppd(root = root,
#                                    df=morpho_data_patient,
#                                    transform = transforms.ToTensor(),
#                                    mesh = "area-001",
#                                    cuttype = "dome",
#                                    crop = False,
#                                    limit = 635,
#                                    points= 2000,
#                                    train= True)
# Aneux_Dataset_save(dataset = Aneuxmodel_Dataset_train_ppd, filepath ='./Datasets/Aneux_Dataset_2000pt_ppd_600train.pt')

# Aneux_Dataset_test_ppd = Aneuxmodel_Dataset_ppd(root = root,
#                                    df=morpho_data_patient,
#                                    transform = transforms.ToTensor(),
#                                    mesh = "area-001",
#                                    cuttype = "dome",
#                                    crop = False,
#                                    limit = 635,
#                                    points= 2000,
#                                    train= False)
# Aneux_Dataset_save(dataset = Aneux_Dataset_test_ppd, filepath ='./Datasets/Aneux_Dataset_2000pt_ppd_100test.pt')

In [15]:
import pandas as pd
from pointnetfunct.PointNet_dataset import Aneux_Dataset_load,Aneux_Dataset_save
from pointnetfunct.PointNet_dataset import Aneuxmodel_Dataset
# df = pd.DataFrame()
# Aneux_Dataset = Aneuxmodel_Dataset(root = root,
#                                    df=morpho_data_patient,
#                                    transform = transforms.ToTensor(),
#                                    mesh = "area-001",
#                                    cuttype = "dome",
#                                    crop = False,
#                                    limit = 635,
#                                    points= 2000,
#                                    train= True)
# Aneux_Dataset_save(dataset = Aneux_Dataset, filepath ='./Datasets/Aneux_Dataset_2000pt_sample_600train.pt')

# Aneux_Dataset_test = Aneuxmodel_Dataset(root = root,
#                                    df=morpho_data_patient,
#                                    transform = transforms.ToTensor(),
#                                    mesh = "area-001",
#                                    cuttype = "dome",
#                                    crop = False,
#                                    limit = 635,
#                                    points= 2000,
#                                    train= False)
# Aneux_Dataset_save(dataset = Aneux_Dataset_test, filepath ='./Datasets/Aneux_Dataset_2000pt_sample_100test.pt')

#Aneux_Dataset = Aneux_Dataset_load('./Datast/Aneux_Dataset_1000pt_fsp_clip.pt')

In [16]:
Aneux_Dataset = Aneux_Dataset_load('./Datasets/Aneux_Dataset_2000pt_sample_600train.pt')
Aneux_Dataset.paraout = True
Aneux_Dataset_test = Aneux_Dataset_load('./Datasets/Aneux_Dataset_2000pt_sample_100test.pt')

d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(['source_x',"cuttype","dataset"], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\pointnetfunct\data_process_ml.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [17]:
import torch
import open3d as o3d
import numpy as np

#torch.save(Aneux_Dataset, './Aneux_Dataset_3000pt_fsp_noclip.pt')

In [18]:
from pointnetfunct.model_loader import vtp_to_point_cloud_cutvessel,vtp_to_point_cloud,vtp_to_point_cloud_cutvessel_fps,vtp_to_point_cloud_fps
modelcov_vessel_vtp = vtp_to_point_cloud("./test_model/ANSYS_UNIGE_09_dome.vtp")
modelcov_vtp = vtp_to_point_cloud("./test_model/ANSYS_UNIGE_09.vtp")
modelcov_vessel_ppd_vtp = vtp_to_point_cloud_fps("./test_model/ANSYS_UNIGE_09_dome.vtp")
modelcov_ppd_vtp = vtp_to_point_cloud_fps("./test_model/ANSYS_UNIGE_09.vtp")
o3d.visualization.draw_geometries([modelcov_vessel_vtp])
o3d.visualization.draw_geometries([modelcov_vtp])
o3d.visualization.draw_geometries([modelcov_vessel_ppd_vtp])
o3d.visualization.draw_geometries([modelcov_ppd_vtp])


In [1]:
from pointnetfunct.model_loader import vtp_to_mesh,vtp_to_point_cloud
import open3d as o3d
#mesh_vessel_vtp = vtp_to_point_cloud("./test_model/SNF00000614.vtp", points= 2000)
mesh_vessel_vtp = vtp_to_mesh("./test_model/ANSYS_UNIGE_09_cut1.vtp")
mesh_vtp = vtp_to_point_cloud("./test_model/SNF00000614_cut1.stl", points= 2000)
o3d.visualization.draw_geometries([mesh_vessel_vtp])
o3d.visualization.draw_geometries([mesh_vtp])

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
print(len(Aneux_Dataset.model_table),len(Aneux_Dataset.label))
print(len(Aneux_Dataset_test.model_table),len(Aneux_Dataset_test.label))

100 100
635 635


In [ ]:
Aneux_Dataset_load(filepath ='./Aneux_Dataset_2000pt_sample_noclip.pt')

d:\Universityofleeds\MSC_PROG\Msc_project\code\data_process_ml.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(morpho_data_cut1.columns[3:23], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\data_process_ml.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morpho_data_cut1.drop(['source_x',"cuttype","dataset"], axis=1, inplace=True)
d:\Universityofleeds\MSC_PROG\Msc_project\code\data_process_ml.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
from torch.utils.data import Subset
torch.manual_seed(30)
train_size = int(len(Aneux_Dataset) * 0.8) # 80% training data
valid_size = len(Aneux_Dataset) - train_size
train_data, valid_data = random_split(Aneux_Dataset, [train_size, valid_size])


train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=20,
    shuffle=True,
    #num_workers=2, 
    pin_memory=True
)

valid_loader = torch.utils.data.DataLoader(
    valid_data,
    batch_size=20, # Forward pass only so batch size can be larger
    shuffle=False,
    #num_workers=2, 
    pin_memory=True
)